## Creating the Dataset 

In [91]:
import numpy as np

np.random.seed(42)
x = np.random.rand(100, 1)
y = 1 + 2 * x + .1 * np.random.randn(100, 1)

# Shuffles the indices
idx = np.arange(100)
np.random.shuffle(idx)

# Uses first 80 random indices for train
train_idx = idx[:80]
# Uses the remaining indices for validation
val_idx = idx[80:]


# Generates train and validation sets
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]
print(x_train.shape)


(80, 1)


### Formatting the Dataset for Pytorch

In [92]:
import torch
import torch.optim as optim
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
x_train_tensor = torch.from_numpy(x_train).float().to(device)
y_train_tensor = torch.from_numpy(y_train).float().to(device)

In [93]:
from model import LayerLinearRegression

In [94]:
# Now we can create a model and send it at once to the device
model = LayerLinearRegression().to(device)
# We can also inspect its parameters using its state_dict
print(model.state_dict())

lr = 1e-1
n_epochs = 1000

loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(n_epochs):
    # What is this?!?
    model.train()

    # No more manual prediction!
    # yhat = a + b * x_tensor
    yhat = model(x_train_tensor)
    
    loss = loss_fn(y_train_tensor, yhat)
    loss.backward()    
    optimizer.step()
    optimizer.zero_grad()
    
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[-0.2971]])), ('linear.bias', tensor([0.2363]))])
OrderedDict([('linear.weight', tensor([[1.9690]])), ('linear.bias', tensor([1.0235]))])


In [95]:
# Testing the prediction 
a = np.array([1.5])
y = model(torch.as_tensor(a,dtype=torch.float32,device=device))
print(y)

tensor([3.9770], grad_fn=<AddBackward0>)


### Saving the Model

In [136]:
torch.save(model.state_dict(),'model.pth')

In [139]:
import os
import sagemaker
import tarfile
print(os.getcwd())

# files = [f for f in os.listdir('.') if os.path.isfile(f)]
# for f in files:
#     print(f)

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
                   
os.makedirs('export',exist_ok=True)
os.makedirs('export/code',exist_ok=True)
os.system('cp model.py export/code/')
os.system('cp generate.py export/code/inference.py')
os.system('cp model.pth export/')
os.system('cp requirements.txt export/code/')
os.chdir('export')
os.system('tar -czvf ../model.tar.gz .')
# make_tarfile('../model.tar.gz','./')
os.chdir('../')

s3_path_to_data  = sagemaker.Session().upload_data(bucket='pahts-test-bucket',
                                              path=os.path.join(os.getcwd(),'model.tar.gz'),
                                              key_prefix='linear_model_key')
# s3_path_to_data  = sagemaker.Session().upload_data(bucket='pahts-test-bucket',
#                                               path=os.path.join(os.getcwd(),'model.py'),
#                                               key_prefix='linear_model_key')
# s3_path_to_data  = sagemaker.Session().upload_data(bucket='pahts-test-bucket',
#                                               path=os.path.join(os.getcwd(),'generate.py'),
#                                               key_prefix='linear_model_key')
# print(s3_path_to_data)

/home/ec2-user/SageMaker


### Exporting the model as an endpoint

In [159]:
from sagemaker.pytorch import PyTorch
from sagemaker.predictor import JSONSerializer, JSONDeserializer, Predictor

class JSONPredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, JSONSerializer, JSONDeserializer)

import sagemaker
from sagemaker.pytorch import PyTorchModel

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
print(bucket)

pytorch_model = PyTorchModel(model_data='s3://pahts-test-bucket/linear_model_key/model.tar.gz',
                     role=role,
                     entry_point='inference.py',
                     framework_version='1.5.0',
                     py_version='py3',
                     source_dir='',
                    predictor_cls=JSONPredictor)

# Deploy model, https://aws.amazon.com/sagemaker/pricing/
predictor = pytorch_model.deploy(instance_type='ml.t2.medium',initial_instance_count=1) #Cheapest

sagemaker-us-east-2-690762613439
---------------------!

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## Evaluate and Test

In [162]:
print('Evaluating')
input = {'xvalue':2}
input_json = json.dumps(input) 

# response = predictor.predict(input_json, initial_args={'ContentType': 'application/json'})
response = predictor.predict(input)
print(response)

The json_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Evaluating
tensor([4.9615])


In [147]:
import requests

url = 'https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/pytorch-inference-2021-02-19-22-17-20-522/invocations'
myobj = {'xvalue': 2}

x = requests.post(url, data = myobj)

print(x.text)

{"message":"Missing Authentication Token"}
